In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [14]:
# Function to convert image to RGB text representation
def image_to_rgb_text(image_path):
    """
    Convert an image to a text representation where each pixel's RGB values
    are converted to strings and connected with semicolons
    
    Args:
        image_path (str): Path to the input image
        
    Returns:
        str: Text representation of the image's RGB values
    """
    # Open and convert image to RGB
    img = Image.open(image_path).convert('RGB')
    # Convert to numpy array
    img_array = np.array(img)
    
    # Initialize empty string to store RGB values
    rgb_text = ""
    
    # Iterate through each row and column
    for row in range(img_array.shape[0]):
        for col in range(img_array.shape[1]):
            # Get RGB values for current pixel
            r, g, b = img_array[row, col]
            # Add RGB values to string, separated by commas
            rgb_text += f"{r},{g},{b}"
            
            # Add semicolon if not the last pixel in the row
            if col < img_array.shape[1] - 1:
                rgb_text += ";"
                
        # Add newline if not the last row
        if row < img_array.shape[0] - 1:
            rgb_text += "\n"
    
    return rgb_text


# Function to convert RGB text back to image
def rgb_text_to_image(rgb_text, output_path):
    """
    Convert text representation of RGB values back to an image
    
    Args:
        rgb_text (str): Text representation of RGB values
        output_path (str): Path to save the output image
        
    Returns:
        numpy.ndarray: Image array
    """
    # Split text into rows
    rows = rgb_text.strip().split('\n')
    
    # Get dimensions
    height = len(rows)
    width = len(rows[0].split(';'))
    
    # Initialize empty array
    img_array = np.zeros((height, width, 3), dtype=np.uint8)
    
    # Fill array with RGB values
    for i, row in enumerate(rows):
        pixels = row.split(';')
        for j, pixel in enumerate(pixels):
            r, g, b = map(int, pixel.split(','))
            img_array[i, j] = [r, g, b]
    
    # Convert array to image and save
    img = Image.fromarray(img_array)
    img.save(output_path)
    
    return img_array

In [ ]:
test_image = Image.open("YET.png")
test_image

In [8]:
test_image_array = np.array(test_image.convert('RGB'))
test_image_array.shape

(1500, 1500, 3)

In [11]:
image_text = image_to_rgb_text("YET.png")

In [13]:
len(image_text)

26087544

In [16]:
open("YET.txt", "w").write(image_text)

26087544

In [15]:
rgb_text_to_image(image_text, "YET_reconstructed.png")

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]

In [18]:
import numpy as np
from PIL import Image
import re
from collections import Counter, defaultdict
import heapq
from typing import Dict, List, Tuple
import base64
import zlib
import json

# 1. Run Length Encoding (RLE)
class RLECompressor:
    @staticmethod
    def compress(text: str) -> str:
        count = 1
        prev = text[0]
        result = []
        
        for char in text[1:]:
            if char == prev:
                count += 1
            else:
                result.extend([str(count), prev])
                count = 1
                prev = char
                
        result.extend([str(count), prev])
        return "".join(result)
    
    @staticmethod
    def decompress(text: str) -> str:
        result = []
        i = 0
        while i < len(text):
            count_str = ""
            while text[i].isdigit():
                count_str += text[i]
                i += 1
            count = int(count_str)
            result.append(text[i] * count)
            i += 1
        return "".join(result)

# 2. Dictionary-based compression (LZ77-like)
class DictionaryCompressor:
    def __init__(self, window_size: int = 1024):
        self.window_size = window_size
        
    def compress(self, text: str) -> List[Tuple[int, int, str]]:
        result = []
        pos = 0
        
        while pos < len(text):
            match = self._find_longest_match(text, pos)
            if match[0] == 0:  # No match found
                result.append((0, 0, text[pos]))
                pos += 1
            else:
                result.append(match)
                pos += match[1]
                
        return result
    
    def _find_longest_match(self, text: str, pos: int) -> Tuple[int, int, str]:
        end = pos + 1
        if pos >= len(text):
            return (0, 0, '')
            
        start = max(0, pos - self.window_size)
        longest_match = (0, 0, text[pos])
        
        while end <= len(text):
            pattern = text[pos:end]
            found_pos = text.find(pattern, start, pos)
            
            if found_pos == -1:
                break
                
            longest_match = (pos - found_pos, len(pattern), 
                           text[pos + len(pattern)] if end < len(text) else '')
            end += 1
            
        return longest_match

# 3. Byte Pair Encoding
class BPECompressor:
    def __init__(self, num_merges: int = 1000):
        self.num_merges = num_merges
        self.encoder = {}
        self.decoder = {}
        
    def get_stats(self, sequences: List[str]) -> Dict[Tuple[str, str], int]:
        pairs = defaultdict(int)
        for seq in sequences:
            symbols = seq.split()
            for i in range(len(symbols) - 1):
                pairs[symbols[i], symbols[i + 1]] += 1
        return pairs
        
    def merge_vocab(self, pair: Tuple[str, str], v_in: List[str]) -> List[str]:
        v_out = []
        bigram = re.escape(' '.join(pair))
        p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
        for word in v_in:
            w_out = p.sub(''.join(pair), word)
            v_out.append(w_out)
        return v_out
        
    def fit(self, text: str):
        # Initialize with character vocabulary
        vocab = [' '.join(text)]
        pairs = self.get_stats(vocab)
        
        for i in range(self.num_merges):
            if not pairs:
                break
            best = max(pairs, key=pairs.get)
            vocab = self.merge_vocab(best, vocab)
            pairs = self.get_stats(vocab)
            
            merged_token = ''.join(best)
            self.encoder[best] = merged_token
            self.decoder[merged_token] = best
            
    def compress(self, text: str) -> str:
        self.fit(text)
        compressed = text
        for pair, merged in self.encoder.items():
            compressed = compressed.replace(''.join(pair), merged)
        return compressed
        
    def decompress(self, compressed: str) -> str:
        text = compressed
        for merged, pair in self.decoder.items():
            text = text.replace(merged, ''.join(pair))
        return text

# 4. Delta Encoding for RGB values
class RGBDeltaCompressor:
    @staticmethod
    def compress(rgb_text: str) -> str:
        lines = rgb_text.split('\n')
        compressed_lines = []
        
        for line in lines:
            pixels = line.split(';')
            compressed_pixels = []
            prev_r, prev_g, prev_b = 0, 0, 0
            
            for pixel in pixels:
                r, g, b = map(int, pixel.split(','))
                # Store differences from previous values
                delta_r = r - prev_r
                delta_g = g - prev_g
                delta_b = b - prev_b
                compressed_pixels.append(f"{delta_r},{delta_g},{delta_b}")
                prev_r, prev_g, prev_b = r, g, b
                
            compressed_lines.append(';'.join(compressed_pixels))
            
        return '\n'.join(compressed_lines)
    
    @staticmethod
    def decompress(compressed: str) -> str:
        lines = compressed.split('\n')
        decompressed_lines = []
        
        for line in lines:
            pixels = line.split(';')
            decompressed_pixels = []
            curr_r, curr_g, curr_b = 0, 0, 0
            
            for pixel in pixels:
                delta_r, delta_g, delta_b = map(int, pixel.split(','))
                curr_r += delta_r
                curr_g += delta_g
                curr_b += delta_b
                decompressed_pixels.append(f"{curr_r},{curr_g},{curr_b}")
                
            decompressed_lines.append(';'.join(decompressed_pixels))
            
        return '\n'.join(decompressed_lines)

# 5. Custom RGB Pattern Tokenizer
class RGBPatternCompressor:
    def __init__(self):
        self.patterns = {}
        self.reverse_patterns = {}
        self.pattern_counter = 0
        
    def _find_common_patterns(self, text: str) -> Dict[str, int]:
        # Find common RGB patterns (e.g., "255,255,255")
        pattern = r'\d+,\d+,\d+'
        patterns = Counter(re.findall(pattern, text))
        return {k: v for k, v in patterns.items() if v > 1}
    
    def compress(self, text: str) -> str:
        common_patterns = self._find_common_patterns(text)
        compressed = text
        
        # Replace common patterns with tokens
        for pattern in common_patterns:
            if pattern not in self.patterns:
                token = f"<{self.pattern_counter}>"
                self.patterns[pattern] = token
                self.reverse_patterns[token] = pattern
                self.pattern_counter += 1
            compressed = compressed.replace(pattern, self.patterns[pattern])
            
        # Store pattern dictionary at the start of compressed string
        header = json.dumps(self.reverse_patterns) + "|||"
        return header + compressed
    
    def decompress(self, compressed: str) -> str:
        # Extract pattern dictionary and compressed data
        header, compressed_data = compressed.split("|||")
        patterns = json.loads(header)
        
        # Replace tokens with original patterns
        decompressed = compressed_data
        for token, pattern in patterns.items():
            decompressed = decompressed.replace(token, pattern)
            
        return decompressed

# Benchmark different compression methods
def benchmark_compression_methods(rgb_text: str):
    original_size = len(rgb_text)
    results = []
    
    # 1. Basic zlib compression
    zlib_compressed = zlib.compress(rgb_text.encode())
    results.append(("zlib", len(zlib_compressed), original_size))
    
    # 2. Delta encoding + zlib
    delta_compressor = RGBDeltaCompressor()
    delta_compressed = delta_compressor.compress(rgb_text)
    delta_zlib = zlib.compress(delta_compressed.encode())
    results.append(("Delta + zlib", len(delta_zlib), original_size))
    
    # 3. Pattern tokenization + zlib
    pattern_compressor = RGBPatternCompressor()
    pattern_compressed = pattern_compressor.compress(rgb_text)
    pattern_zlib = zlib.compress(pattern_compressed.encode())
    results.append(("Pattern + zlib", len(pattern_zlib), original_size))
    
    # 4. BPE + zlib
    bpe_compressor = BPECompressor(num_merges=500)
    bpe_compressed = bpe_compressor.compress(rgb_text)
    bpe_zlib = zlib.compress(bpe_compressed.encode())
    results.append(("BPE + zlib", len(bpe_zlib), original_size))
    
    # Print results
    print(f"\nOriginal size: {original_size} bytes")
    print("\nCompression Results:")
    for method, compressed_size, orig_size in results:
        ratio = (compressed_size / orig_size) * 100
        print(f"{method:15} {compressed_size:8d} bytes  {ratio:6.2f}% of original")

# Test the compression methods
if __name__ == "__main__":
    # Run benchmarks
    print("Running compression benchmarks...")
    benchmark_compression_methods(image_text)

Running compression benchmarks...

Original size: 26087544 bytes

Compression Results:
zlib               91044 bytes    0.35% of original
Delta + zlib       73542 bytes    0.28% of original
Pattern + zlib     40241 bytes    0.15% of original
BPE + zlib         91044 bytes    0.35% of original
